In [58]:
from tqdm import tqdm
import math
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import json
import re
import spacy
sp = spacy.load('en_core_web_sm')
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import statistics

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/oliverwang/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
f = open('./labelled_sentences.json')
data = json.load(f)
f.close

In [ ]:
data[0]['Label']['classifications'][0]['answer'][0]['value']

In [ ]:
data[0]['Labeled Data']

In [ ]:
punctuation = re.compile('[^\w\s]')
space = re.compile('  +')

clean_data = []
for i in range(len(data)):
    if bool(data[i]['Label']):


        sentence = sp(space.sub(' ', punctuation.sub('', data[i]['Labeled Data'])))
        




        if data[i]['Label']['classifications'][0]['answer'][0]['value'] == 'neutral':
            clean_data.append({'sentence': sentence, 'label': 0})
        elif data[i]['Label']['classifications'][0]['answer'][0]['value'] == 'negative':
            clean_data.append({'sentence': sentence, 'label': -1})
        else:
            clean_data.append({'sentence': sentence, 'label': 1})
    
clean_data[0]

In [ ]:
sentences_lemmaed = []
for sentence in clean_data:
    words = []
    for w in sentence['sentence']:
        words.append(w.lemma_.lower())
    sentences_lemmaed.append([" ".join(words), sentence['label']])
sentences_lemmaed[2]

In [ ]:
words = set()

for par in tqdm(sentences_lemmaed):
    for w in par[0].split():
        words.add(w)

words = list(words)
        
len(words)

In [ ]:
class_sentence_count = {-1: 0, 0: 0, 1: 0}

for par in tqdm(sentences_lemmaed):
    class_sentence_count[par[1]] += 1


In [ ]:
word_class_sentence_count = {}

for word in tqdm(words):
    word_class_sentence_count[word] = {-1: 0, 0: 0, 1: 0}
    for par in sentences_lemmaed:
        if word in par[0].split():
            word_class_sentence_count[word][par[1]] += 1


In [ ]:
word_sentence_count = {}
for k, v in word_class_sentence_count.items():
    word_sentence_count[k] = sum(v.values())

In [ ]:
total = len(sentences_lemmaed)
total

In [ ]:
pmi = {}

# p(w|c)
for word in tqdm(words):
    pmi[word] = {}
    
    if word_class_sentence_count[word][-1] != 0:
        pmi[word][-1] = math.log(
            (word_class_sentence_count[word][-1] / total) / 
            (class_sentence_count[-1] / total))
    else:
        pmi[word][-1] = math.log(
            (1 / total) / 
            (class_sentence_count[-1] / total))


    if word_class_sentence_count[word][0] != 0:
        pmi[word][0] = math.log(
            (word_class_sentence_count[word][0] / total) / 
            (class_sentence_count[0] / total))
    else:
        pmi[word][0] = math.log(
            (1 / total) / 
            (class_sentence_count[0] / total))

    
        
    if word_class_sentence_count[word][1] != 0:
        pmi[word][1] = math.log(
            (word_class_sentence_count[word][1] / total) / 
            (class_sentence_count[1] / total))
    else:
        pmi[word][1] = math.log(
            (1 / total) / 
            (class_sentence_count[1] / total))

In [ ]:
S = {}

for word, values in pmi.items():
    S[word] = values[1] - values[-1]

In [ ]:
s_df = pd.DataFrame.from_dict(S, orient='index')
s_df.columns = ['sent']
freq = pd.DataFrame.from_dict(word_sentence_count, orient='index')
freq.columns = ['freq']
s_df = s_df.join(freq)
s_df.sort_values('sent', ascending=False)

In [ ]:
s_df.loc['negative']

In [ ]:
vader_lexicon_file = './vader_lexicon.txt'
vader_lexicon = {}
with open(vader_lexicon_file, 'r') as f:
    for line in f.readlines():
        line = line.strip()
        vader_lexicon[line.split('\t')[0]] = float(line.split('\t')[1])
vader_words = list(vader_lexicon.keys())

s_new = s_df[~s_df.index.isin(vader_words)].copy()
s_com = s_df[s_df.index.isin(vader_words)].copy()

s_com['vader'] = s_com.apply(lambda x: vader_lexicon.get(x.name), axis=1)


In [ ]:
s_com_filter = s_com[s_com['freq']>15].copy().reset_index()

In [54]:
px.scatter(s_com_filter, x='vader', y='sent', text='index', size='freq',
           labels={'vader': 'Vader score', 'sent': 'Customized score'}, 
           title='Comparison of Lexicon')

In [ ]:
thres = 100
s_new_filter = s_new[s_new['freq']>thres].copy().reset_index()
s_new_filter

In [ ]:
px.scatter(s_new_filter, x='freq', y='sent', text='index')

In [56]:
update_dict = {
    # existing
    "crisis": 0,
    "positive": -2,
    "great": 0,
    "authority": -2,
    "death": -1,
    "die": -1.5,
    "bad": -1,
    "fear": -1.5,
    "unemployment": -0.5,
    "emergency": 0,
    "fight": 1,
    "stop": 0.5,
    "loser": -0.5,
    "restriction": -0.5,
    "risk": 0,
    "cut": 0,
    "low":0,
    "number": -1,
    "care": -0.5,
    "increase": -0.5,
    "united": -0.5,

    
    "Crisis": 0,
    "Positive": -1.5,
    # "Great": 0,
    # "Authority": -2,
    
    # new
    "coronavirus": -1,
    "pandemic": 0.5,
    "outbreak": -1.5,
    "virus": -1,
    "lockdown": -1,
    "trump": -1.5,
    
    "Coronavirus": -1,
    "Pandemic": 0.5,
    "Outbreak": -1.5,
    "Virus": -1,
    "Lockdown": -1,
    "Trump": -1.5,
    "TRUMP": -1.5,
    "covid19": -1
}

## Calculate Sentence Scores

In [59]:
sid = SentimentIntensityAnalyzer()
sid.lexicon.update(update_dict)

In [80]:
sentence_scores = []
labels = []
for i in range(len(data)):
    if bool(data[i]['Label']):
        sentence = data[i]['Labeled Data']
        sentence_scores.append(sid.polarity_scores(sentence)['compound'])
        label = data[i]['Label']['classifications'][0]['answer'][0]['value']
        if label == 'neutral':
            labels.append(0)
        elif label == 'negative':
            labels.append(-1)
        else:
            labels.append(1)


In [85]:
import numpy as np
from accuracy_metrics import *

def find_opt_metrics(predictions, gold_labels, lower_bound=-1, upper_bound=1, step=0.05):
    
    # opt_lower = -0.75
    # opt_upper = 0.75
    max_f1 = 0 
    opt_metrics = None
    for lower in np.arange(lower_bound, upper_bound, step):
        for upper in np.arange(upper_bound, lower, step*-1):
            metrics = AccuracyMetrics(predictions, gold_labels, neutral_upper=upper, neutral_lower=lower)
            if metrics.macro_f1 > max_f1:
                # opt_lower = lower
                # opt_upper = upper
                max_f1 = metrics.macro_f1
                opt_metrics = metrics
    
    return opt_metrics

In [87]:
opt_metrics = find_opt_metrics(sentence_scores, labels)

In [90]:
opt_neutral_lower = opt_metrics.neutral_upper
opt_neutral_upper = opt_metrics.neutral_upper

In [94]:
import csv
def write_opt_acc_metrics(accuracy_metrics, path):

    with open(path, mode='w') as csv_file:
        accuracy_writer = csv.writer(csv_file, delimiter=',', quotechar='"')
        accuracy_writer.writerow(['aggregation_method'] + list(accuracy_metrics.result_dict().keys()) + ['neutral_lower', 'neutral_upper'])

        for key in accuracy_metricskeys():
            this_met = accuracy_metrics
            accuracy_writer.writerow([key] + list(this_met.result_dict().values()) + [str(this_met.neutral_lower), str(this_met.neutral_upper)])

In [96]:
write_opt_acc_metrics(opt_metrics, '../results/sentence_only_acc_metrics.csv')

NameError: name 'accuracy_metrics_dict' is not defined